In [5]:
import os
import asyncio
from typing import Any
from dataclasses import dataclass

import nest_asyncio
nest_asyncio.apply

from openai import AsyncOpenAI
from pydantic_ai.models.openai import OpenAIModel

import streamlit as st
from pydantic_ai import Agent, RunContext
from pydantic import BaseModel, Field, ValidationError

from dotenv import load_dotenv
_ = load_dotenv()


In [6]:
Client = AsyncOpenAI(
    base_url= "http://localhost:11434/v1",
    api_key="not needed"
)

model = OpenAIModel('llama3.1 8b', openai_client=Client)

In [7]:
TAVILIY_API_KEY = os.getenv("TAVILY_API_KEY")

if not TAVILIY_API_KEY:
    raise ValueError("TAVILIY_API_KEY must be set in the environment variables.")

In [8]:
# Regular python dataclass, lacks some additional features only available in pydantic basemodel
@dataclass
class SearchDataclass:
    max_results: int
    todays_date: str

@dataclass
class ResearchDepedencies:
    todays_date: str

# This is a pydantic basemodel
class ResearchResult(BaseModel):
    research_title: str = Field(description='Markdown title header of the report')
    research_main: str = Field(description='Main body section of the report')
    research_bullets: str = Field(description='Set of bullet points summarising keypoints')

In [15]:
# Create the Agent
search_agent = Agent(
    model,
    result_type=ResearchResult,  # result is usually a pydantic basemodel
    deps_type = ResearchDepedencies,
    system_prompt=  'You are a helpful research assistant, and expert in research. '
                    'When given a query, you will identify good keywords to do 3-5 searches using the provided search tool'
                    'Then combine the results into a detailed response.'   # static system prompt
)

# Agent -- Dynamic System Prompt (used at run time)
# Dynamic system prompts: depends on context that isn't known until runtime, 
# defined via functions decorated with @agent.system_prompt
@search_agent.system_prompt
async def add_current_date(ctx: RunContext[ResearchDepedencies]) -> str:
    todays_date = ctx.deps.todays_date
    system_prompt = (
        f"You're a helpful research assistant and an expert in research. "
        f"When given a question, write strong keywords to do 3-5 searches in total "
        f"(each with a query_number) and then combine the results. "
        f"If you need today's date it is {todays_date}. "
        f"Focus on providing accurate and current information."
    )
    return system_prompt

# Agent -- Tool
@search_agent.tool
async def get_search(ctx: RunContext[SearchDataclass], query:str, query_number:int) -> dict[str,Any]:
    """Get the search for a keyword query.
    Args:
        query: keywords to search.
    """
    max_results = ctx.deps.max_results
    results = await tavily_client.get_search_context(query=query, max_results=max_results)
    return results

In [ ]:
st.set_page_config(page_title="AI Researcher", layout="centered")
st.title("LLM News Researcher")
st.write("stay updated on the latest trends")

# User input section
st.sidebar.title("Seach Query")
query = st.sidebar.text_input("Enter your query:", value="example text here")
max_result = st.sidebar.slider("Number of searches: ", min_value=3, max_value=8, values=3)

st.write("Use the slide bard to adjust search parameters")
if st.button("Get latest news!"):
    with st.spinner("AI is thinking, please wait ..."):
        result_data = asyncio.run(do_search(query,max_result))
    
    st.markdown(result_data.resarch_title)
    st.markdown(f"<div style='line-height:1.6;'>{result_data.research_main}</div>", unsafe_allow_html=True)